# Experiment 003: Rotation Tightening (fix_direction)

Implement rotation tightening from saspav kernel to minimize bounding box by finding optimal rotation angle for each configuration.

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import json

getcontext().prec = 25
scale_factor = 1  # No scaling for now

print("Imports done")

Imports done


In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree."""
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        
        initial_polygon = Polygon([
            (float(Decimal('0.0')), float(tip_y)),
            (float(top_w / Decimal('2')), float(tier_1_y)),
            (float(top_w / Decimal('4')), float(tier_1_y)),
            (float(mid_w / Decimal('2')), float(tier_2_y)),
            (float(mid_w / Decimal('4')), float(tier_2_y)),
            (float(base_w / Decimal('2')), float(base_y)),
            (float(trunk_w / Decimal('2')), float(base_y)),
            (float(trunk_w / Decimal('2')), float(trunk_bottom_y)),
            (float(-(trunk_w / Decimal('2'))), float(trunk_bottom_y)),
            (float(-(trunk_w / Decimal('2'))), float(base_y)),
            (float(-(base_w / Decimal('2'))), float(base_y)),
            (float(-(mid_w / Decimal('4'))), float(tier_2_y)),
            (float(-(mid_w / Decimal('2'))), float(tier_2_y)),
            (float(-(top_w / Decimal('4'))), float(tier_1_y)),
            (float(-(top_w / Decimal('2'))), float(tier_1_y)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))
    
    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def get_tree_list_side_length(tree_list):
    """Get the side length of the bounding box for a list of trees."""
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1])))

def get_total_score(dict_of_side_length):
    """Calculate total score from side lengths."""
    score = Decimal('0')
    for k, v in dict_of_side_length.items():
        score += v ** 2 / Decimal(str(k))
    return score

def parse_csv(csv_path):
    """Parse submission CSV into tree lists."""
    result = pd.read_csv(csv_path)
    result['x'] = result['x'].str.strip('s')
    result['y'] = result['y'].str.strip('s')
    result['deg'] = result['deg'].str.strip('s')
    result[['group_id', 'item_id']] = result['id'].str.split('_', n=2, expand=True)
    
    dict_of_tree_list = {}
    dict_of_side_length = {}
    for group_id, group_data in result.groupby('group_id'):
        tree_list = [ChristmasTree(center_x=row['x'], center_y=row['y'], angle=row['deg'])
                     for _, row in group_data.iterrows()]
        dict_of_tree_list[group_id] = tree_list
        dict_of_side_length[group_id] = get_tree_list_side_length(tree_list)
    
    return dict_of_tree_list, dict_of_side_length

print("Helper functions defined")

Helper functions defined


In [4]:
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find optimal rotation angle for a configuration."""
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points)
    
    # Get convex hull points (faster to optimize)
    hull_points = points_np[ConvexHull(points_np).vertices]
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, 89.999), 
        method='bounded'
    )
    found_angle_deg = res.x
    found_side = res.fun
    
    improvement = initial_side - found_side
    EPSILON = 1e-10
    
    if improvement > EPSILON:
        best_angle_deg = found_angle_deg
        best_side = Decimal(str(found_side))
    else:
        best_angle_deg = 0.0
        best_side = Decimal(str(initial_side))
    
    return best_side, best_angle_deg

def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees in configuration."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]
    
    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])
    
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    
    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center
    
    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(Decimal(str(trees[i].angle)) + Decimal(str(angle_deg)))
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print("Rotation functions defined")

Rotation functions defined


In [5]:
# Test on small N first (MANDATORY per strategy)
print("Testing on small N values first...")
print("="*50)

# Load baseline
baseline_path = '/home/nonroot/snapshots/santa-2025/21328309254/submission/submission.csv'
dict_of_tree_list, dict_of_side_length = parse_csv(baseline_path)

current_score = get_total_score(dict_of_side_length)
print(f"Current total score: {current_score:.6f}")
print()

# Test on N = 10, 20, 30, 50, 100
test_ns = [10, 20, 30, 50, 100]
for n in test_ns:
    group_id = f'{n:03d}'
    trees = dict_of_tree_list[group_id]
    old_side = dict_of_side_length[group_id]
    
    best_side, best_angle = optimize_rotation(trees)
    improvement = old_side - best_side
    
    print(f"N={n:3d}: old={float(old_side):.6f}, new={float(best_side):.6f}, "
          f"improvement={float(improvement):.6f}, angle={best_angle:.2f}°")

Testing on small N values first...


Current total score: 70.647327

N= 10: old=1.940696, new=1.940696, improvement=0.000000, angle=0.00°
N= 20: old=2.742469, new=2.742469, improvement=0.000000, angle=0.00°
N= 30: old=3.290365, new=3.290365, improvement=0.000000, angle=0.00°
N= 50: old=4.247076, new=4.247076, improvement=0.000000, angle=0.00°
N=100: old=5.878187, new=5.878187, improvement=0.000000, angle=0.00°


In [6]:
# Apply fix_direction to ALL configurations
print("\nApplying fix_direction to all 200 configurations...")
print("="*50)

improved_count = 0
total_improvement = Decimal('0')

for n in range(1, 201):
    group_id = f'{n:03d}'
    trees = dict_of_tree_list[group_id]
    old_side = dict_of_side_length[group_id]
    
    best_side, best_angle = optimize_rotation(trees)
    
    if best_side < old_side:
        improvement = old_side - best_side
        total_improvement += improvement
        improved_count += 1
        
        # Apply rotation
        fixed_trees = apply_rotation(trees, best_angle)
        dict_of_tree_list[group_id] = fixed_trees
        dict_of_side_length[group_id] = best_side
        
        if n <= 20 or n % 50 == 0:
            print(f"N={n:3d}: {float(old_side):.6f} -> {float(best_side):.6f} "
                  f"(improvement={float(improvement):.6f}, angle={best_angle:.2f}°)")

print(f"\nImproved {improved_count} configurations")
print(f"Total side length improvement: {float(total_improvement):.6f}")


Applying fix_direction to all 200 configurations...



Improved 2 configurations
Total side length improvement: 0.000001


In [7]:
# Calculate new total score
new_score = get_total_score(dict_of_side_length)
score_improvement = current_score - new_score

print(f"\n{'='*50}")
print(f"Original score: {float(current_score):.6f}")
print(f"New score:      {float(new_score):.6f}")
print(f"Improvement:    {float(score_improvement):.6f}")
print(f"{'='*50}")
print(f"\nTarget: 68.888293")
print(f"Gap to target: {float(new_score) - 68.888293:.6f}")


Original score: 70.647327
New score:      70.647327
Improvement:    0.000000

Target: 68.888293
Gap to target: 1.759034


In [8]:
# Check for overlaps in improved configurations
print("\nValidating no overlaps...")

def check_overlaps(trees, tolerance=1e-12):
    if len(trees) <= 1:
        return []
    polygons = [t.polygon for t in trees]
    overlaps = []
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx > i:
                if polygons[i].intersects(polygons[idx]) and not polygons[i].touches(polygons[idx]):
                    intersection = polygons[i].intersection(polygons[idx])
                    if intersection.area > tolerance:
                        overlaps.append((i, idx, intersection.area))
    return overlaps

overlap_configs = []
for n in range(1, 201):
    group_id = f'{n:03d}'
    trees = dict_of_tree_list[group_id]
    overlaps = check_overlaps(trees)
    if overlaps:
        overlap_configs.append((n, len(overlaps)))

print(f"Configurations with overlaps: {len(overlap_configs)}")
if overlap_configs:
    print(f"First 5: {overlap_configs[:5]}")


Validating no overlaps...


Configurations with overlaps: 0


In [9]:
# Save submission if improved and no overlaps
if float(score_improvement) > 0 and len(overlap_configs) == 0:
    print("\nSaving improved submission...")
    
    tree_data = []
    for group_name in sorted(dict_of_tree_list.keys(), key=lambda x: int(x)):
        tree_list = dict_of_tree_list[group_name]
        for item_id, tree in enumerate(tree_list):
            tree_data.append({
                'id': f'{group_name}_{item_id}',
                'x': f's{tree.center_x}',
                'y': f's{tree.center_y}',
                'deg': f's{tree.angle}'
            })
    
    df_out = pd.DataFrame(tree_data)
    df_out.to_csv('/home/code/experiments/003_fix_direction/submission.csv', index=False)
    df_out.to_csv('/home/submission/submission.csv', index=False)
    print(f"Saved {len(df_out)} rows")
    print(f"First 5 rows:")
    print(df_out.head())
else:
    print("\nNo improvement or has overlaps - keeping baseline")
    import shutil
    shutil.copy('/home/nonroot/snapshots/santa-2025/21328309254/submission/submission.csv', 
                '/home/submission/submission.csv')
    shutil.copy('/home/nonroot/snapshots/santa-2025/21328309254/submission/submission.csv',
                '/home/code/experiments/003_fix_direction/submission.csv')


Saving improved submission...
Saved 20100 rows
First 5 rows:
      id                     x                      y                  deg
0  001_0                  s0.0   s-0.3000000000000007  s44.999999999999744
1  002_0   s0.1540970696213643  s-0.03854074269478543  s203.62937773065684
2  002_1  s-0.1540970696213643   s-0.5614592573052146  s23.629377730656792
3  003_0   s0.1857706481046834   s-0.5474488423355981    s111.125132292893
4  003_1   s0.2961705278065425  s-0.05255115766440198     s66.370622269343


In [ ]:
# Save metrics
metrics = {
    'cv_score': float(new_score),
    'original_score': float(current_score),
    'improvement': float(score_improvement),
    'improved_configs': improved_count,
    'overlap_configs': len(overlap_configs),
    'target': 68.888293,
    'gap': float(new_score) - 68.888293
}

with open('/home/code/experiments/003_fix_direction/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nMetrics saved: {metrics}")